In [1]:
#Preventing aspect: 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
df_churn = pd.read_csv("WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [4]:
df_churn.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [2]:
np.random.seed(42)
y = df_churn['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)  # Encode target as binary
X = df_churn.drop('Churn',axis = 1)
xTrain,xTest,yTrain,yTest = train_test_split(X,y,test_size = 0.7)

In [3]:
from sklearn.linear_model import LogisticRegression
X = pd.get_dummies(X, drop_first=True)

# Split data into training and testing sets
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.7, random_state=42)

# Train logistic regression model
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(xTrain, yTrain)

# Print model accuracy on test set
print("Model accuracy on test set:", log_reg.score(xTest, yTest))

Model accuracy on test set: 0.8087609004258771


In [6]:
from gurobipy import Model, GRB, quicksum


In [4]:
pip install gurobipy==11.0.0


   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.2 MB 2.6 MB/s eta 0:00:04
   - -------------------------------------- 0.4/10.2 MB 5.5 MB/s eta 0:00:02
   --- ------------------------------------ 0.9/10.2 MB 7.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/10.2 MB 7.9 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/10.2 MB 8.7 MB/s eta 0:00:01
   --------- ------------------------------ 2.5/10.2 MB 9.5 MB/s eta 0:00:01
   ------------ --------------------------- 3.2/10.2 MB 10.2 MB/s eta 0:00:01
   --------------- ------------------------ 3.9/10.2 MB 11.0 MB/s eta 0:00:01
   ----------------- ---------------------- 4.5/10.2 MB 11.1 MB/s eta 0:00:01
   -------------------- ------------------- 5.3/10.2 MB 11.7 MB/s eta 0:00:01
   ----------------------- ---------------- 6.0/10.2 MB 12.0 MB/s eta 0:00:01
   -------------------------- ------------- 6.8/10.2 MB 12.4 MB/s eta 0:00:01


In [10]:
n_features = xTrain.shape[1]
n_samples = xTrain.shape[0]

# Create Gurobi model
model = Model("RobustLogisticRegression")
beta = model.addVars(n_features, lb=-GRB.INFINITY, name="beta")

# Define the objective using a direct approximation for log-loss
log_losses = []
for i in range(n_samples):
    y_i = yTrain.iloc[i]
    x_i = xTrain.iloc[i].values
    
    # Linear combination: beta * x_i
    lin_comb = quicksum(beta[j] * x_i[j] for j in range(n_features))
    
    # Introduce an auxiliary variable for the negative log-likelihood
    log_loss = model.addVar(lb=0, name=f"log_loss_{i}")
    
    # Add constraint: log_loss >= max(0, -y_i * lin_comb)
    model.addConstr(log_loss >= 0)
    model.addConstr(log_loss >= -y_i * lin_comb)
    
    # Approximate log(1 + exp(-y_i * (x_i * beta))) using piecewise linear constraints if necessary
    
    log_losses.append(log_loss)

# Objective: Minimize the sum of log-losses
model.setObjective(quicksum(log_losses), GRB.MINIMIZE)

# Optimize the model
model.optimize()

# Extract optimized coefficients
optimized_beta = np.array([beta[j].X for j in range(n_features)])
print("Optimized beta coefficients:", optimized_beta)

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4224 rows, 15713 columns and 10388 nonzeros
Model fingerprint: 0xe1b062b2
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 4224 rows and 15713 columns
Presolve time: 0.03s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   1.091000e-03      0s
Extra simplex iterations after uncrush: 537
     537    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 537 iterations and 0.06 seconds (0.01 work units)
Optimal objective  0.000000000e+00
Optimized beta coefficients: [0. 0. 0. ... 0. 0. 0.]


In [17]:
import gurobipy as gp